In [ ]:
import boto3
import json
import time

# Initialize clients
iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

# Define variables (same as in creation script)
suffix = f"{region}-{account_id}"
agent_name = "hr-assistant-function-def"
agent_bedrock_allow_policy_name = f"{agent_name}-ba-{suffix}"
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'
agent_alias_name = f"{agent_name}-alias"
lambda_function_role = f'{agent_name}-lambda-role-{suffix}'
lambda_function_name = f'{agent_name}-{suffix}'

# Set your agent details
agent_id = 'CDL0KSVUWE'  # Your agent ID
agent_alias_id = 'SY5ZUHIIQ2'  # Your agent alias ID
action_group_id = 'YTNOJR6BN1'  # Your action group ID

In [ ]:
# Step 1: Delete action group
try:
    print("Deleting action group...")
    bedrock_agent_client.delete_agent_action_group(
        agentId=agent_id,
        agentVersion='DRAFT',
        actionGroupId=action_group_id
    )
    print("Action group deleted successfully")
except Exception as e:
    print(f"Error deleting action group: {e}")

In [ ]:
# Step 2: Delete agent alias
try:
    print("Deleting agent alias...")
    bedrock_agent_client.delete_agent_alias(
        agentAliasId=agent_alias_id,
        agentId=agent_id
    )
    print("Agent alias deleted successfully")
except Exception as e:
    print(f"Error deleting agent alias: {e}")

In [ ]:
# Step 3: Delete agent
try:
    print("Deleting agent...")
    bedrock_agent_client.delete_agent(
        agentId=agent_id
    )
    print("Agent deleted successfully")
except Exception as e:
    print(f"Error deleting agent: {e}")

In [7]:
# Step 4: Delete Lambda function
try:
    print("Deleting Lambda function...")
    lambda_client.delete_function(
        FunctionName=lambda_function_name
    )
    print("Lambda function deleted successfully")
except Exception as e:
    print(f"Error deleting Lambda function: {e}")

# Step 5: Clean up IAM roles and policies
try:
    print("Cleaning up IAM resources...")
    
    # Detach policies from roles
    try:
        iam_client.detach_role_policy(
            RoleName=agent_role_name, 
            PolicyArn=f'arn:aws:iam::{account_id}:policy/{agent_bedrock_allow_policy_name}'
        )
    except Exception as e:
        print(f"Error detaching agent policy: {e}")
    
    try:
        iam_client.detach_role_policy(
            RoleName=lambda_function_role, 
            PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
        )
    except Exception as e:
        print(f"Error detaching Lambda policy: {e}")
    
    # Delete roles
    for role_name in [agent_role_name, lambda_function_role]:
        try:
            iam_client.delete_role(RoleName=role_name)
            print(f"Deleted role: {role_name}")
        except Exception as e:
            print(f"Error deleting role {role_name}: {e}")
    
    # Delete custom policy
    try:
        iam_client.delete_policy(
            PolicyArn=f'arn:aws:iam::{account_id}:policy/{agent_bedrock_allow_policy_name}'
        )
        print(f"Deleted policy: {agent_bedrock_allow_policy_name}")
    except Exception as e:
        print(f"Error deleting policy: {e}")
        
    print("Cleanup completed!")
    
except Exception as e:
    print(f"Error during IAM cleanup: {e}")

Deleting Lambda function...
Error deleting Lambda function: An error occurred (ResourceNotFoundException) when calling the DeleteFunction operation: Function not found: arn:aws:lambda:us-east-1:447924746755:function:hr-assistant-function-def-us-east-1-447924746755
Cleaning up IAM resources...
Error detaching agent policy: An error occurred (NoSuchEntity) when calling the DetachRolePolicy operation: The role with name AmazonBedrockExecutionRoleForAgents_hr-assistant-function-def cannot be found.
Error detaching Lambda policy: An error occurred (NoSuchEntity) when calling the DetachRolePolicy operation: The role with name hr-assistant-function-def-lambda-role-us-east-1-447924746755 cannot be found.
Error deleting role AmazonBedrockExecutionRoleForAgents_hr-assistant-function-def: An error occurred (NoSuchEntity) when calling the DeleteRole operation: The role with name AmazonBedrockExecutionRoleForAgents_hr-assistant-function-def cannot be found.
Error deleting role hr-assistant-function